# Phase 2 
Antoine Demers-Bergeron et Joseph Thompson

## Partie 1
### 1. Structure de données pour les composantes connexes
On a implémenté une structure identique aux graphes. On veut que les composantes connexes aient les même propriétés que les graphes.

In [43]:

include("../phase1/node.jl")
include("../phase1/edges.jl")
include("../phase1/graph.jl")
include("../phase1/read_stsp.jl")
include("comp_connexes.jl")
include("../phase1/main.jl")
include("queue.jl")

last_in_first_out_priority_queue

In [44]:
mutable struct Comp_Connexe{Y,T} <: AbstractGraph{T}
    name::String
    nodes::Vector{Node{Y}}
    edges::Vector{Edge{T,Y}}
  end
  
  """Ajoute un noeud à la composante connexe."""
  function add_node!(graph::Comp_Connexe{Y,T}, node::Node{Y}) where {Y,T}
    push!(graph.nodes, node)
    graph
  end
  
  """Ajoute une arête à la composante connexe"""
  function add_edge!(graph::Comp_Connexe{Y,T}, edge::Edge{T,Y}) where {Y,T}
    push!(graph.edges, edge)
    if !(edge.node1.name in nodes_names(graph))
      add_node!(graph, edge.node1)
      @warn "Le noeud",edge.node1.name," n'était pas dans le graphe. Il a été ajouté."
    end
    if !(edge.node2.name in nodes_names(graph))
      add_node!(graph, edge.node2)
      @warn "Le noeud",edge.node2.name," n'était pas dans le graphe. Il a été ajouté."
    end
    graph
  end
  
  """Affiche la composante connexe"""
  function show(graph::Comp_Connexe)
    println("La composante connexe ", name(graph), " a ", nb_nodes(graph), " noeuds.")
    for node in nodes(graph)
      show(node)
    end
    println("La composante connexe ", name(graph), " a ", nb_edges(graph), " arretes.")
    for edge in edges(graph)
      show(edge)
    end
  end

Base.show

### 2. Algorithme de Kruskal


In [45]:
"""
Ce code implémente l'algorithme de Kruskal qui crée un arbre de
recouvrement minimum à partir d'un graphe
"""

""" Tri par insertion une liste d'arêtes selon leur poids """
function insertion!(A::Vector{Edge{Int,Vector{Float64}}}) 
    n = length(A)
    for j = 2 : n
        key = A[j].weight
        edge = A[j]
        i = j - 1
        while i > 0 && A[i].weight > key
            A[i+1] = A[i]
            i = i - 1
        end
        A[i+1] = edge
    end
    A
end

""" Vérifie si les deux noeud d'une arête donnée sont dans la même composante connexe """
function NodesInSameCConnexe(edge::Edge,CConnexes)
    node1, node2 = nodes(edge)
    for C in CConnexes
        if node1 in C.nodes
            if node2 in C.nodes
                return true
            else return false
            end
        end
    end

end

""" Prend une arête [s_i,s_j] et la liste des composantes connexes en input et réunit les composantes connexes de s_i et s_j  """
function Merge_CConnexes!(edge, CConnexes)
    node1, node2 = nodes(edge)
    indice1, indice2 = 0, 0
    for i in eachindex(CConnexes)
        C = CConnexes[i]
        if node1 in nodes(C)  ##Assigne l'indice de la composante connexe du noeud 1
            indice1 = i     
        end
        if node2 in nodes(C)  ####Assigne l'indice de la composante connexe du noeud 2
            indice2 = i
        end
    end

    if indice1 != 0 && indice2 != 0
        append!(CConnexes[indice1].nodes, CConnexes[indice2].nodes)
        append!(CConnexes[indice1].edges, CConnexes[indice2].edges)
        add_edge!(CConnexes[indice1],edge)
        deleteat!(CConnexes, indice2)
    end

    return CConnexes
end

""" Construit l'arbre de recouvrement minimum avec l'algorithme de Kruskal"""
function Kruskal(graphe::Graph)
    A = insertion!(graphe.edges)   ##Tri les arêtes selon leur poids
    CConnexes_list = []
    i=1
    for node in nodes(graphe)   ##Crée une composante connexe pour chaque noeud
        push!(CConnexes_list,Comp_Connexe(string(i),Node{Vector{Float64}}[node],Edge{Int,Vector{Float64}}[]))
    end
    
    for edge in A
        if length(CConnexes_list) == 1
            break       ##On arrête lorsqu'il reste une seule composante connexe
        end
        if (NodesInSameCConnexe(edge,CConnexes_list)) != true    
            CConnexes_list = Merge_CConnexes!(edge,CConnexes_list) ##Réunit les composantes connexes des deux noeuds
        end
    end
    return CConnexes_list[1]
end

Kruskal

On a ensuite testé l'algorithme sur le graphe présenté en exemple dans les notes de laboratoires.

In [46]:
graphe_test = Graph("Test",Node{Vector{Float64}}[],Edge{Int,Vector{Float64}}[])

#Nodes 
nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
nodef = Node("f",[0.])
nodeg = Node("g",[0.])
nodeh = Node("h",[0.])
nodei = Node("i",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee, nodef, 
                nodeg, nodeh, nodei]

#Edges
edge1 = Edge(nodea,nodeb, 4)
edge2 = Edge(nodea,nodeh, 8)
edge3 = Edge(nodeb, nodeh, 11)
edge4 = Edge(nodeb, nodec, 8)
edge5 = Edge(nodeh, nodei, 7)
edge6 = Edge(nodeh, nodeg, 1)
edge7 = Edge(nodeg, nodei, 6)
edge8 = Edge(nodeg, nodef, 2)
edge9 = Edge(nodec, nodef, 4)
edge10 = Edge(nodei, nodec, 2)
edge11 = Edge(nodef, nodee, 10)
edge12 = Edge(nodee, noded, 9)
edge13 = Edge(nodec,noded, 7)
edge14 = Edge(noded, nodef, 14)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14]

##Construction du graphe
for i in node_list
    add_node!(graphe_test,i)
end
for i in edge_list
    add_edge!(graphe_test, i)
end

CConnexe = Kruskal(graphe_test)

show(CConnexe)

La composante connexe 1 a 9 noeuds.
Node e, data: [0.0]
Node a, data: [0.0]
Node b, data: [0.0]
Node i, data: [0.0]
Node c, data: [0.0]
Node h, data: [0.0]
Node g, data: [0.0]
Node f, data: [0.0]
Node d, data: [0.0]
La composante connexe 1 a 8 arretes.
Arête de a à b, poids: 4
Arête de i à c, poids: 2
Arête de h à g, poids: 1
Arête de g à f, poids: 2
Arête de c à f, poids: 4
Arête de c à d, poids: 7
Arête de a à h, poids: 8
Arête de e à d, poids: 9


### 3. Test sur diverses instances de TSP symétriques

In [47]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/bays29.tsp","graphe1")
CConnexe = Kruskal(graphe1)

show(CConnexe)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
La composante connexe 1 a 29 noeuds.
Node 7, data: [1650.0, 650.0]
Node 23, data: [1840.0, 1240.0]
Node 11, data: [840.0, 550.0]
Node 3, data: [40.0, 2090.0]
Node 5, data: [750.0, 2030.0]
Node 9, data: [790.0, 2260.0]
Node 26, data: [490.0, 2130.0]
Node 29, data: [360.0, 1980.0]
Node 6, data: [1030.0, 2070.0]
Node 12, data: [1170.0, 2300.0]
Node 1, data: [1150.0, 1760.0]
Node 28, data: [1260.0, 1910.0]
Node 2, data: [630.0, 1660.0]
Node 21, data: [830.0, 1770.0]
Node 16, data: [1280.0, 1200.0]
Node 8, data: [1490.0, 1630.0]
Node 24, data: [1260.0, 1500.0]
Node 27, data: [1460.0, 1420.0]
Node 4, data: [750.0, 1100.0]
Node 15, data: [750.0, 900.0]
Node 10, data: [710.0, 1310.0]
Node 20, data: [590.0, 1390.0]
Node 14, data: [510.0, 700.0]
Node 18, data: [460.0, 860.0]
Node 22, data: [490.0, 500.0]
Node 19, data: [1040.0, 950.0]
Node 13, data: [970.0, 1340.0]
Node 17, data: [230.0, 590.0]
Node 25, data: [1280.0, 790.0]
La comp

In [48]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/swiss42.tsp","graphe1")
CConnexe = Kruskal(graphe1)

show(CConnexe)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
La composante connexe 1 a 42 noeuds.
Node 25, data: [0.0, 0.0]
Node 18, data: [0.0, 0.0]
Node 32, data: [0.0, 0.0]
Node 15, data: [0.0, 0.0]
Node 17, data: [0.0, 0.0]
Node 16, data: [0.0, 0.0]
Node 38, data: [0.0, 0.0]
Node 9, data: [0.0, 0.0]
Node 10, data: [0.0, 0.0]
Node 24, data: [0.0, 0.0]
Node 42, data: [0.0, 0.0]
Node 12, data: [0.0, 0.0]
Node 13, data: [0.0, 0.0]
Node 19, data: [0.0, 0.0]
Node 6, data: [0.0, 0.0]
Node 27, data: [0.0, 0.0]
Node 3, data: [0.0, 0.0]
Node 28, data: [0.0, 0.0]
Node 4, data: [0.0, 0.0]
Node 5, data: [0.0, 0.0]
Node 29, data: [0.0, 0.0]
Node 1, data: [0.0, 0.0]
Node 2, data: [0.0, 0.0]
Node 7, data: [0.0, 0.0]
Node 30, data: [0.0, 0.0]
Node 31, data: [0.0, 0.0]
Node 11, data: [0.0, 0.0]
Node 26, data: [0.0, 0.0]
Node 8, data: [0.0, 0.0]
Node 14, data: [0.0, 0.0]
Node 20, data: [0.0, 0.0]
Node 33, data: [0.0, 0.0]
Node 23, data: [0.0, 0.0]
Node 39, data: [0.0, 0.0]
Node 22, data: [0.0, 0.0

## Partie 2 : Implémentation des deux heuristiques



In [49]:
tree_root = Tree("1", 0)
child1 = Tree("2", 1)
child2 = Tree("3", 2)
child3 = Tree("4", 3)
child4 = Tree("5", 4)
first_row = [ child1, child2, child3]
second_row = [child4]
for child in first_row
    change_parent!(child, tree_root)
end
for child in second_row
    change_parent!(child, child3)
end

In [50]:
show(tree_root)

Node 1 has  rank 0and 3 children.
listing of children : 
node_visited: 2     
 its parent is 1     
 its rank (or distance to parent) is 0
node_visited: 3     
 its parent is 1     
 its rank (or distance to parent) is 0
node_visited: 4     
 its parent is 1     
 its rank (or distance to parent) is 0
child: 5
node_visited: 5     
 its parent is 4     
 its rank (or distance to parent) is 0


In [51]:
"""Trouve la racine d'un arbre """
function find_root(tree::Tree{T}) where T
  #root is the only node without a parent
  if isnothing(parent(tree))
      return tree
  else
      return find_root(parent(tree))
  end
end


"""Implementation du premier heuristic"""
function rank_union!(tree_1::Tree{T}, tree_2::Tree{T}) where T
    root_1 = find_root(tree_1)
    root_2 = find_root(tree_2)
    if rank(root_1) == rank(root_2)
      change_parent!(root_2, root_1)
      change_rank!(root_1, rank(root_1) + 1)
      return root_1
    elseif rank(root_1) > rank(root_2)
      change_parent!(root_2, root_21)
      return root_1
    else
      change_parent!(root_1, root_2)
      return root_2
    end
end

rank_union!

In [52]:
tree_A = Tree("A", 0)
tree_B = Tree("B", 1)
new_tree = rank_union!(tree_A, tree_B)
tree_C = Tree("C", 2)
tree_D = Tree("D", 3)
second_tree = rank_union!(tree_C, tree_D)
final_tree = rank_union!(new_tree, second_tree)
show(final_tree)


Node A has  rank 2and 2 children.
listing of children : 
node_visited: B     
 its parent is A     
 its rank (or distance to parent) is 0
node_visited: C     
 its parent is A     
 its rank (or distance to parent) is 1
child: D
node_visited: D     
 its parent is C     
 its rank (or distance to parent) is 0


In [53]:


"""Implementation du deuxieme heuristic"""
function path_compression(tree::Tree{T}) where T
    #la racine est le seul noeud sans parent
    if isnothing(parent(tree))
        return tree
    else
        root =  path_compression(parent(tree))
        change_parent!(tree, root)
        return root
    end
end

path_compression

## Partie 3 : Algorithme de Prim

In [54]:

"""crée une liste d'adjacence a partir d'un graphe"""
function adjacency_dict( graph::Graph , priority_queue::PriorityQueue{PriorityItem{Tree{T}}}) where T
    #Dictionaire de adjacence
    adj_dict = Dict()
    #Dictionaire de correspondance entre les noeuds et les items de la file de priorite
    correspondance_dict = Dict()
    #Chaque noeud est une clef du dictionaire de adjacence
    for (node, priority_item) in zip(nodes(graph),priority_queue.items)
        adj_dict[priority_item] = Dict()
        correspondance_dict[node] = priority_item
    end
    #Ajoute les voisins de chaque noeud dans le dictionaire de adjacence
    for edge in edges(graph)
        node_1, node_2 = nodes(edge)
        prior_item_1 = correspondance_dict[node_1]
        prior_item_2 = correspondance_dict[node_2]
        adj_dict[prior_item_1][prior_item_2] = weight(edge)
        adj_dict[prior_item_2][prior_item_1] = weight(edge)
    end
    return adj_dict
end

"""Crée une file de priorite a partir d'un graphe. Le noeud de depart a une priorite de 0 et les autres ont une priorite de Inf"""
function prims_priority_queue(graph::Graph{Y,T}, start_node_name::String) where {Y,T}
    priority_queue = PriorityQueue{PriorityItem{Tree{Y}}}()
    for node in nodes(graph)
        if name(node) == start_node_name
            blank_tree = Tree(name(node), data(node))
            priority_item = PriorityItem( 0 , blank_tree)
        else
            blank_tree = Tree(name(node), data(node))
            priority_item = PriorityItem( Inf, blank_tree)
        end
        push!(priority_queue, priority_item)
    end
    return priority_queue
end

function prims_algorithm(graph::Graph{Y,T}; start_node_name::Any = nothing) where {Y,T}
    #initialisation
    if isnothing(start_node_name)
        start_node_name = name(nodes(graph)[1])
    end
    #initialisation de la file de priorite et du dictionaire d'adjacence
    priority_queue = prims_priority_queue(graph, start_node_name)
    adjacency_list = adjacency_dict(graph, priority_queue)
    #sauvegarde de la racine
    root = poplast!(priority_queue)
    priority_node = root
    #boucle principale
    while !is_empty(priority_queue)
        for  (neighbor, edge_weight) in adjacency_list[priority_node]
            for item in priority_queue.items
                if name(data(neighbor)) == name(data(item))
                    if  edge_weight < priority(neighbor)
                        change_parent!(data(neighbor), data(priority_node))
                        change_rank!(data(neighbor), edge_weight)
                        priority!(neighbor, edge_weight)
                    end
                break
                end
            end
        end
        priority_node = poplast!(priority_queue)         
    end
    return data(root)
end

prims_algorithm (generic function with 1 method)

In [55]:
"""Convert un arbre en graphe"""
function tree_to_graph(tree::Tree{T}) where T
  graph = Graph(name(tree), Node{T}[], Edge{Float64, T}[])
  nodes_to_visit = copy(children(tree))
  while length(nodes_to_visit) != 0
    current_tree = popfirst!(nodes_to_visit)
    node = Node(name(current_tree), data(current_tree))
    parent_tree= parent(current_tree)
    for child in children(current_tree)
      push!(nodes_to_visit, child)
    end
    if !isnothing(parent_tree)
      parent_node = Node(name(parent_tree), data(parent_tree))
      distance = convert(Float64, rank(current_tree))
      edge = Edge(parent_node, node, distance)
      add_edge!(graph, edge)
    end
  end
  graph
end

tree_to_graph

In [56]:

test_tree = prims_algorithm(graphe_test,start_node_name =  "a")
test_graph = tree_to_graph(test_tree)
show(test_graph)

Graph a has 9 nodes.
Node a, data: [0.0]
Node b, data: [0.0]
Node c, data: [0.0]
Node f, data: [0.0]
Node d, data: [0.0]
Node i, data: [0.0]
Node g, data: [0.0]
Node e, data: [0.0]
Node h, data: [0.0]
Graph a has 8 edges.
Arête de a à b, poids: 4.0
Arête de b à c, poids: 8.0
Arête de c à f, poids: 4.0
Arête de c à d, poids: 7.0
Arête de c à i, poids: 2.0
Arête de f à g, poids: 2.0
Arête de d à e, poids: 9.0
Arête de g à h, poids: 1.0


┌ Warning: ("Le noeud", "a", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:35
┌ Warning: ("Le noeud", "b", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "c", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "f", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "d", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "i", " n'était pas da

In [57]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/bays29.tsp","graphe1")

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓


(Graph{Vector{Float64}, Int64}("graphe1", Node{Vector{Float64}}[Node{Vector{Float64}}("1", [1150.0, 1760.0]), Node{Vector{Float64}}("2", [630.0, 1660.0]), Node{Vector{Float64}}("3", [40.0, 2090.0]), Node{Vector{Float64}}("4", [750.0, 1100.0]), Node{Vector{Float64}}("5", [750.0, 2030.0]), Node{Vector{Float64}}("6", [1030.0, 2070.0]), Node{Vector{Float64}}("7", [1650.0, 650.0]), Node{Vector{Float64}}("8", [1490.0, 1630.0]), Node{Vector{Float64}}("9", [790.0, 2260.0]), Node{Vector{Float64}}("10", [710.0, 1310.0])  …  Node{Vector{Float64}}("20", [590.0, 1390.0]), Node{Vector{Float64}}("21", [830.0, 1770.0]), Node{Vector{Float64}}("22", [490.0, 500.0]), Node{Vector{Float64}}("23", [1840.0, 1240.0]), Node{Vector{Float64}}("24", [1260.0, 1500.0]), Node{Vector{Float64}}("25", [1280.0, 790.0]), Node{Vector{Float64}}("26", [490.0, 2130.0]), Node{Vector{Float64}}("27", [1460.0, 1420.0]), Node{Vector{Float64}}("28", [1260.0, 1910.0]), Node{Vector{Float64}}("29", [360.0, 1980.0])], Edge{Int64, Vect

In [58]:
bays_29_tree = prims_algorithm(graphe1)

Tree{Vector{Float64}}("1", [1150.0, 1760.0], nothing, Tree{Vector{Float64}}[Tree{Vector{Float64}}("28", [1260.0, 1910.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[Tree{Vector{Float64}}("6", [1030.0, 2070.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[Tree{Vector{Float64}}("12", [1170.0, 2300.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[], 55), Tree{Vector{Float64}}("5", [750.0, 2030.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[Tree{Vector{Float64}}("26", [490.0, 2130.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[Tree{Vector{Float64}}("29", [360.0, 1980.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[Tree{Vector{Float64}}("3", [40.0, 2090.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[], 77)], 39)], 57), Tree{Vector{Float64}}("9", [790.0, 2260.0], Tree{Vector{F

In [59]:
bays_29_graph = tree_to_graph(bays_29_tree)


┌ Warning: ("Le noeud", "1", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:35
┌ Warning: ("Le noeud", "28", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "21", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "6", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "2", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:39
┌ Warning: ("Le noeud", "12", " n'était pas

Graph{Vector{Float64}, Float64}("1", Node{Vector{Float64}}[Node{Vector{Float64}}("1", [1150.0, 1760.0]), Node{Vector{Float64}}("28", [1260.0, 1910.0]), Node{Vector{Float64}}("21", [830.0, 1770.0]), Node{Vector{Float64}}("6", [1030.0, 2070.0]), Node{Vector{Float64}}("2", [630.0, 1660.0]), Node{Vector{Float64}}("12", [1170.0, 2300.0]), Node{Vector{Float64}}("5", [750.0, 2030.0]), Node{Vector{Float64}}("20", [590.0, 1390.0]), Node{Vector{Float64}}("26", [490.0, 2130.0]), Node{Vector{Float64}}("9", [790.0, 2260.0])  …  Node{Vector{Float64}}("16", [1280.0, 1200.0]), Node{Vector{Float64}}("25", [1280.0, 790.0]), Node{Vector{Float64}}("14", [510.0, 700.0]), Node{Vector{Float64}}("27", [1460.0, 1420.0]), Node{Vector{Float64}}("7", [1650.0, 650.0]), Node{Vector{Float64}}("17", [230.0, 590.0]), Node{Vector{Float64}}("22", [490.0, 500.0]), Node{Vector{Float64}}("24", [1260.0, 1500.0]), Node{Vector{Float64}}("8", [1490.0, 1630.0]), Node{Vector{Float64}}("23", [1840.0, 1240.0])], Edge{Float64, Vect

In [60]:
show(bays_29_tree)

Node 1 has  rank 0and 2 children.
listing of children : 
node_visited: 28     
 its parent is 1     
 its rank (or distance to parent) is 45
child: 6
node_visited: 21     
 its parent is 1     
 its rank (or distance to parent) is 65
child: 2
node_visited: 6     
 its parent is 28     
 its rank (or distance to parent) is 60
child: 12
child: 5
node_visited: 2     
 its parent is 21     
 its rank (or distance to parent) is 42
child: 20
node_visited: 12     
 its parent is 6     
 its rank (or distance to parent) is 55
node_visited: 5     
 its parent is 6     
 its rank (or distance to parent) is 61
child: 26
child: 9
node_visited: 20     
 its parent is 2     
 its rank (or distance to parent) is 67
child: 10
node_visited: 26     
 its parent is 5     
 its rank (or distance to parent) is 57
child: 29
node_visited: 9     
 its parent is 5     
 its rank (or distance to parent) is 46
node_visited: 10     
 its parent is 20     
 its rank (or distance to parent) is 28
child: 13
child: 4

In [61]:
bays_29_tree.children

2-element Vector{Tree{Vector{Float64}}}:
 Tree{Vector{Float64}}("28", [1260.0, 1910.0], Tree{Vector{Float64}}("1", [1150.0, 1760.0], nothing, Tree{Vector{Float64}}[#= circular reference @-3 =#], 0), Tree{Vector{Float64}}[Tree{Vector{Float64}}("6", [1030.0, 2070.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[Tree{Vector{Float64}}("12", [1170.0, 2300.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[], 55), Tree{Vector{Float64}}("5", [750.0, 2030.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[Tree{Vector{Float64}}("26", [490.0, 2130.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[Tree{Vector{Float64}}("29", [360.0, 1980.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[Tree{Vector{Float64}}("3", [40.0, 2090.0], Tree{Vector{Float64}}(#= circular reference @-3 =#), Tree{Vector{Float64}}[], 77)], 39)], 57), Tree{Vector{Float64}}("9", [790.0

In [62]:
show(bays_29_graph)

Graph 1 has 29 nodes.
Node 1, data: [1150.0, 1760.0]
Node 28, data: [1260.0, 1910.0]
Node 21, data: [830.0, 1770.0]
Node 6, data: [1030.0, 2070.0]
Node 2, data: [630.0, 1660.0]
Node 12, data: [1170.0, 2300.0]
Node 5, data: [750.0, 2030.0]
Node 20, data: [590.0, 1390.0]
Node 26, data: [490.0, 2130.0]
Node 9, data: [790.0, 2260.0]
Node 10, data: [710.0, 1310.0]
Node 29, data: [360.0, 1980.0]
Node 13, data: [970.0, 1340.0]
Node 4, data: [750.0, 1100.0]
Node 3, data: [40.0, 2090.0]
Node 15, data: [750.0, 900.0]
Node 19, data: [1040.0, 950.0]
Node 11, data: [840.0, 550.0]
Node 18, data: [460.0, 860.0]
Node 16, data: [1280.0, 1200.0]
Node 25, data: [1280.0, 790.0]
Node 14, data: [510.0, 700.0]
Node 27, data: [1460.0, 1420.0]
Node 7, data: [1650.0, 650.0]
Node 17, data: [230.0, 590.0]
Node 22, data: [490.0, 500.0]
Node 24, data: [1260.0, 1500.0]
Node 8, data: [1490.0, 1630.0]
Node 23, data: [1840.0, 1240.0]
Graph 1 has 28 edges.
Arête de 1 à 28, poids: 45.0
Arête de 1 à 21, poids: 65.0
Arête 